In [2]:
import pandas as pd #csv파일을 이용하기 위해
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### 추천 시스템 구현 과정

1. csv 파일 읽어오기(유저의 평가/운동 영상 정보 or 운동 종류)
2. csv파일을 합쳐 테이블 만들기
3. 아이템 기반 협업 필터링에 맞게 운동-유저 형식으로 변경
4. 코사인 유사도 검사
5. 유사한 영화 추천 받음

### 데이터 셋 로드 및 출력

In [63]:
#user = pd.read_csv('./data/user.csv')
#user.head()

In [3]:
video_data = pd.read_csv('./data/health_video_old.csv')
video_data.head()

,videoId,category,title,difficulty
0,1,leg,초보자도 할 수 있는 다리 운동,0
1,2,shoulder,어깨 스트레칭,0
2,3,leg,고강도 다리 운동,2
3,4,shoulder,넓은 어깨 만들기,1
4,5,thigh,허벅지 하루만 따라하세요,2


In [4]:
print(user.shape,video_data.shape)

NameError: name 'user' is not defined

In [5]:
tfidf_vector = TfidfVectorizer()
#tfidf_vector = TfidfVectorizer(ngram_range=(1,2))
#합치지 않으면 오류가 생겼음?
tfidf_matrix = tfidf_vector.fit_transform(video_data['category'] ).toarray()
#tfidf_matrix = tfidf_vector.fit_transform(video_data)
tfidf_matrix_feature = tfidf_vector.get_feature_names_out()

In [6]:
tfidf_matrix.shape

(10, 5)

In [7]:
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index = video_data.title)
print(tfidf_matrix.shape)
tfidf_matrix.head()

(10, 5)


,abs,eight,leg,shoulder,thigh
title,,,,,
초보자도 할 수 있는 다리 운동,0.0,0.0,1.0,0.0,0.0
어깨 스트레칭,0.0,0.0,0.0,1.0,0.0
고강도 다리 운동,0.0,0.0,1.0,0.0,0.0
넓은 어깨 만들기,0.0,0.0,0.0,1.0,0.0
허벅지 하루만 따라하세요,0.0,0.0,0.0,0.0,1.0


### 영상 제목으로 추천

In [8]:
cosine_sim = cosine_similarity(tfidf_matrix)

In [74]:
cosine_sim.shape

(10, 10)

In [75]:
#타이틀-타이틀 표로 변환
cosine_sim_df = pd.DataFrame(cosine_sim, index = video_data.title, columns = video_data.title)
print(cosine_sim_df.shape)
cosine_sim_df.head()

(10, 10)


title,초보자도 할 수 있는 다리 운동,어깨 스트레칭,고강도 다리 운동,넓은 어깨 만들기,허벅지 하루만 따라하세요,서서하는 복근운동,팔뚝살 돌려깎기,직각 어깨 만들기,어깨가 편해지는 기적의 운동,허벅지 안쪽살 제거!
title,,,,,,,,,,
초보자도 할 수 있는 다리 운동,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
어깨 스트레칭,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
고강도 다리 운동,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
넓은 어깨 만들기,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
허벅지 하루만 따라하세요,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [76]:
#matrix에서 target_title 행을 가져와 정렬하고, k만큼 출력한다.
def title_recommendations(target_title, matrix, items, k=2):
    recom_idx = matrix.loc[:, target_title].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    recom_title = items.iloc[recom_idx, :].title.values
    recom_category = items.iloc[recom_idx, :].category.values
    recom_difficulty = items.iloc[recom_idx, :].difficulty.values
    recom_id = items.iloc[recom_idx, :].id.values
    target_title_list = np.full(len(range(k)), target_title)
    target_category_list = np.full(len(range(k)), items[items.title == target_title].category.values)
    target_difficulty_list = np.full(len(range(k)), items[items.title == target_title].difficulty.values)
    d = {
        'target_title':target_title_list,
        'target_category':target_category_list,
        'target_difficulty':target_difficulty_list,
        'recom_title' : recom_title,
        'recom_category' : recom_category,
        'recom_dificulty' : recom_difficulty,
        'recom_id' : recom_id
    }
    return pd.DataFrame(d)

#제목으로 추천하기- 사용자가 보았던 것과 비슷한 것으로
title_recommendations('초보자도 할 수 있는 다리 운동', cosine_sim_df, video_data)

,target_title,target_category,target_difficulty,recom_title,recom_category,recom_dificulty,recom_id
0,초보자도 할 수 있는 다리 운동,leg,0,초보자도 할 수 있는 다리 운동,leg,0,1
1,초보자도 할 수 있는 다리 운동,leg,0,허벅지 안쪽살 제거!,thigh,2,10


### 카테고리로 추천하기

In [9]:
#타이틀-카테고리 표로 변환
cosine_sim_df2 = pd.DataFrame(cosine_sim, index = video_data.title, columns = video_data.category)
print(cosine_sim_df2.shape)
cosine_sim_df2.head()

(10, 10)


category,leg,shoulder,leg,shoulder,thigh,abs,eight,shoulder,shoulder,thigh
title,,,,,,,,,,
초보자도 할 수 있는 다리 운동,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
어깨 스트레칭,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
고강도 다리 운동,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
넓은 어깨 만들기,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
허벅지 하루만 따라하세요,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [14]:
#matrix에서 target_title 행을 가져와 정렬하고, k만큼 출력한다.
def category_recommendations(target_category, matrix, items, k=2):
    recom_idx = matrix.loc[:, target_category].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    recom_title = items.iloc[recom_idx, :].title.values
    recom_category = items.iloc[recom_idx, :].category.values
    recom_difficulty = items.iloc[recom_idx, :].difficulty.values
    recom_id = items.iloc[recom_idx, :].videoId.values
    #target_title_list = np.full(len(range(k)), items[items.category == target_category].title.values)
    target_category_list = np.full(len(range(k)), target_category)
    #target_difficulty_list = np.full(len(range(k)), items[items.category == target_category].difficulty.values)
    d = {
        #'target_title':target_title_list,
        'target_category':target_category_list,
        #'target_difficulty':target_difficulty_list,
        'recom_title' : recom_title,
        'recom_category' : recom_category,
        'recom_dificulty' : recom_difficulty,
        'recom_id' : recom_id
    }
    return pd.DataFrame(d)

#카테고리로 추천하기- 사용자가 부족한 운동으로
content_based_result =category_recommendations('leg', cosine_sim_df2, video_data)


In [28]:
results_title = content_based_result.iloc[:,1]
print("추천 영상은 다음과 같습니다: ",end=" ")
for title in results_title:
    print(title,end=", ")

추천 영상은 다음과 같습니다:  어깨 스트레칭, 허벅지 하루만 따라하세요, 

In [45]:
#한 항목을 기준으로 반환하는 것이 아니라 2개 이상을 고려해서 반환할 수 있나?
#추천된 반환 목록에서 유저 프로필 정보를 이용해서 순위를 재 배치 할 수 있나?

### 두 개 항목으로 콘텐츠 기반 추천하기

In [46]:
difficulty_data=list(map(str,video_data['difficulty']))
video_data['features'] = video_data['category'] + ' ' + difficulty_data


# compute the TF-IDF matrix of the combined text column
tfidf_vectorizer2 = TfidfVectorizer()
tfidf_matrix2 = tfidf_vectorizer2.fit_transform(video_data['features'])

# compute the cosine similarity matrix
cosine_sim5 = cosine_similarity(tfidf_matrix2)

In [47]:
cosine_sim_df5 = pd.DataFrame(cosine_sim5, index = video_data.title, columns = video_data.features)
print(cosine_sim_df5.shape)
cosine_sim_df5.head()

(10, 10)


features,leg 0,shoulder 0,leg 2,shoulder 1,thigh 2,abs 1,eight 2,shoulder 1,shoulder 0,thigh 2
title,,,,,,,,,,
초보자도 할 수 있는 다리 운동,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
어깨 스트레칭,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
고강도 다리 운동,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
넓은 어깨 만들기,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
허벅지 하루만 따라하세요,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [48]:
def multi_criteria_recommendations(target_category, target_difficulty, matrix, items, k=2):
    # get the indices of videos that match the target category and difficulty
    target_idx = items[(items['category']==target_category) & (items['difficulty']==target_difficulty)].index

    # compute the average cosine similarity of the target videos with all other videos
    sim_scores = np.mean(matrix[target_idx,:], axis=0)

    # get the indices of top-k videos with highest similarity
    recom_idx = sim_scores.argsort()[::-1][:k]

    # get the title, category, difficulty, and id of the recommended videos
    recom_title = items.iloc[recom_idx]['title'].values
    recom_category = items.iloc[recom_idx]['category'].values
    recom_difficulty = items.iloc[recom_idx]['difficulty'].values
    recom_id = items.iloc[recom_idx]['id'].values

    # create a DataFrame of the recommendations and return it
    d = {
        'target_category': np.full(k, target_category),
        'target_difficulty': np.full(k, target_difficulty),
        'recom_title' : recom_title,
        'recom_category' : recom_category,
        'recom_difficulty' : recom_difficulty,
        'recom_id' : recom_id
    }
    return pd.DataFrame(d)

multi_criteria_recommendations('leg', '0', cosine_sim, video_data)

c:\ProgramData\Anaconda3\envs\poseDance\lib\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\ProgramData\Anaconda3\envs\poseDance\lib\site-packages\numpy\core\_methods.py:182: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


,target_category,target_difficulty,recom_title,recom_category,recom_difficulty,recom_id
0,leg,0,허벅지 안쪽살 제거!,thigh,2,10
1,leg,0,어깨가 편해지는 기적의 운동,shoulder,0,9
